# Quality measures tutorial

This is a tutorial for creating quality measures from raw data (.asc). It will include the absolute values and percentage of data for different parameters (missing pupil data, eye-tracking data outside the experiment).



In [ ]:
import csv
from pathlib import Path

import polars as pl

import pymovements as pm

After importing some basic libraries let's load an example eyetracking file.

In [ ]:
asc = "ch1hr007.asc"

data, metadata = pm.gaze.from_asc(
    asc,
    patterns=[
        r"start_recording_(?P<trial>(?:PRACTICE_)?trial_\d+)_(?P<screen>.+)",
        {"pattern": r"stop_recording_", "column": "trial", "value": None},
        {"pattern": r"stop_recording_", "column": "screen", "value": None},
        {
            "pattern": r"start_recording_(?:PRACTICE_)?trial_\d+_page_\d+",
            "column": "activity",
            "value": "reading",
        },
        {
            "pattern": r"start_recording_(?:PRACTICE_)?trial_\d+_question_\d+",
            "column": "activity",
            "value": "question",
        },
        {
            "pattern": r"start_recording_(?:PRACTICE_)?trial_\d+_(familiarity_rating_screen_\d+|subject_difficulty_screen)",
            "column": "activity",
            "value": "rating",
        },
        {"pattern": r"stop_recording_", "column": "activity", "value": None},
        {
            "pattern": r"start_recording_PRACTICE_trial_",
            "column": "practice",
            "value": True,
        },
        {
            "pattern": r"start_recording_trial_",
            "column": "practice",
            "value": False,
        },
        {"pattern": r"stop_recording_", "column": "practice", "value": None},
    ],
)
data.frame

print(data)

We will split the pixel column into two for x and y coordinates of the gaze.

In [ ]:
# Unnest the 'pixel' column

data.frame = data.frame.select(
    [
        pl.all().exclude("pixel"),
        pl.col("pixel").list.get(0).alias("pixel_x"),
        pl.col("pixel").list.get(1).alias("pixel_y"),
    ]
)
data.frame

# Extracting quality measures

The following function is meant to check if the sampling rate of the eyetracker ever deviated from the expected value.
It checks if consecutive timepoints ever differ by more than the value of the expected_diff argument. We're checking only the rows when a task definied by activity_id is performed.
An eyetracker with a constant refresh rate should return 0 skipped_time_absolute and a 0 skipped_time_ratio.


In [ ]:
# function calculating for skipped time


def time_loss(df: pl.DataFrame, task_column: str = 'activity', activity_id: str = 'page',
              target_column: str = 'time', expected_diff: float = 0.5, tolerance: float = 1e-7):
    # Filter the DataFrame for rows where the 'activity' column contains the word 'page'
    filtered_df = df.filter(pl.col(task_column).str.contains(activity_id))
    # Calculate the difference between consecutive rows
    differences = filtered_df[target_column].diff().drop_nulls()
    # Store difference between timestep and expected_diff, where the difference is signifficant
    large_differences = differences.filter(differences > expected_diff) - expected_diff
    # total skipped time
    skipped_time_absolute = sum(large_differences)
    # ratio of skipped time to experiment duration
    total_duration = (df[target_column][len(df) - 1] - df[target_column][0])
    skipped_time_ratio = (skipped_time_absolute / total_duration)
    return skipped_time_absolute, skipped_time_ratio


time_loss(data.frame)

In [ ]:
def missing_pupil(df, sampling_rate, pupil_col):
    miss_pupil_tuple = df[pupil_col].value_counts().row(by_predicate=(pl.col(pupil_col) == 0.0))
    abs_miss_pupil = miss_pupil_tuple[1] / sampling_rate
    per_miss_pupil = miss_pupil_tuple[1] / (df.height)
    return per_miss_pupil, abs_miss_pupil

In [ ]:
def missing_gaze(df, sampling_rate, gaze_x_col):
    abs_miss_gaze_x = data.frame.select(pl.col(gaze_x_col).is_null().sum()).item()
    per_miss_gaze_x = abs_miss_gaze_x / (df.height)
    abs_miss_gaze_x / sampling_rate
    return per_miss_gaze_x, abs_miss_gaze_x

In [ ]:
def off_task_time(df, sampling_rate, data_col):
    null_values_tab = df.null_count()
    abs_miss_screen = null_values_tab[data_col][0]
    per_miss_screen = abs_miss_screen / (df.height)
    return abs_miss_screen / 1000, per_miss_screen

The following function gets the information about validation, specifically average value of all of the validations and maximal value of all of the validations. It is called in the next get_qual_check function.

In [ ]:
def get_validation_data(validations):
    sum_average = 0.0
    max_values = []
    for validation in validations:
        sum_average += float(validation['validation_score_avg'])
        max_values.append(float(validation['validation_score_max']))
    average_average = sum_average / len(validations)
    global_max = max(max_values)
    return average_average, global_max

The following function extracts certain signifficant quality measures from the metadata and makes use of the functions above to calculate its own measures. The result is saved as a csv file.

In [ ]:
def get_qual_check(
        df: pl.DataFrame,  # data frame with raw values
        metadata: dict,  # dictionary with metadata
        csv_name: str = 'out.csv',  # name of the output csv file, need to end with .csv
        pupil_col: str = "pupil",  # column in df where pupil data are stored
        data_col: str = 'screen',  # column in df where the screen activity is stored
        gaze_x_col: str = 'pixel_x',  # column in df with the gaze X coordinates
        trial_col: str = 'trial'):  # column in df with the trial runs

    measures_dict = {}
    # check metadata values
    measures_dict['sampling_rate'] = metadata['sampling_rate']
    measures_dict['data_loss_ratio'] = metadata['data_loss_ratio']
    measures_dict['data_loss_ratio_blinks'] = metadata['data_loss_ratio_blinks']
    measures_dict['total_recording_duration_sec'] = metadata['total_recording_duration_ms'] / 1000

    # Check amount of pupil omissions
    measures_dict['missing_pupil_ratio'], measures_dict['missing_pupil_sec'] = missing_pupil(
        df, measures_dict['sampling_rate'], pupil_col)

    # Check amount of missing gaze data
    measures_dict['missing_gaze_ratio'], measures_dict['missing_gaze_sec'] = missing_gaze(
        df, measures_dict['sampling_rate'], gaze_x_col)

    # Check the amount of time spent not on experimental tasks

    measures_dict['off_task_time_sec'], measures_dict['off_task_time_ratio'] = off_task_time(
        df, metadata['sampling_rate'], data_col)

    # Check the average quality of validation
    measures_dict['average_validation_score'], measures_dict['global_max_validation_score'] = get_validation_data(
        metadata['validations'])

    # Check time loss
    measures_dict['time_loss_sec'] = time_loss(df)[0]
    measures_dict['time_loss_ratio'] = time_loss(df)[1]

    # Divide data frame by trials
    list_of_trials_raw = data.frame.partition_by(by=trial_col)
    list_of_trials = [i for i in list_of_trials_raw if i.item(1, trial_col) is not None]
    i = 0

    # Check the quality measures for separate trials
    for trial in list_of_trials:
        null_ratio_expr = pm.measure.measures.null_ratio("pixel_x", pl.Float64)
        null_ratio = trial.select([null_ratio_expr]).item()
        trial_name = str(trial.item(1, trial_col))
        measures_dict[trial_name + '_null_ratio'] = null_ratio
        measures_dict[trial_name +
                      '_average_validation'] = metadata['validations'][i]["validation_score_avg"]
        measures_dict[trial_name +
                      '_max_validation'] = metadata['validations'][i]["validation_score_max"]
        measures_dict[trial_name + '_error'] = metadata['validations'][i]["error"]
        measures_dict[trial_name + '_tracked_eye'] = metadata['validations'][i]["tracked_eye"]
        i += 1

    # Save measures in csv
    with open(csv_name, "w", newline="") as f:
        w = csv.DictWriter(f, measures_dict.keys())
        w.writeheader()
        w.writerow(measures_dict)
    print(measures_dict)


get_qual_check(data.frame, metadata)